In [1]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd

In [2]:
IMAGE_SIZE = [224, 224] # image size 224 x 224
path="../orange_dataset/"

In [3]:
train = []
i = 0
data_size = 50
for file in os.listdir(os.path.join(path, 'train/fresh')):
    #if(i >= data_size): # take less datas to test about data size
    #    break
    #i+=1
    train.append(['train/fresh/{}'.format(file), 'fresh'])
i = 0
for file in os.listdir(os.path.join(path, 'train/rotten')):
    #if(i >= data_size): # take less datas to test about data size
    #    break
    #i+=1
    train.append(['train/rotten/{}'.format(file), 'rotten'])
train = pd.DataFrame(train, columns=['file', 'class_id'])
train = train.sample(frac=1).reset_index(drop=True)

test = []
for file in os.listdir(os.path.join(path, 'test/fresh')):
    test.append(['test/fresh/{}'.format(file), 'fresh'])
for file in os.listdir(os.path.join(path, 'test/rotten')):
    test.append(['test/rotten/{}'.format(file), 'rotten'])
test = pd.DataFrame(test, columns=['file', 'class_id'])
test = test.sample(frac=1).reset_index(drop=True)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
# create generator for more datas
datagen = ImageDataGenerator(rescale = 1./255,
                             shear_range = 0.2,
                             zoom_range = 0.2,
                             horizontal_flip = True,
                             vertical_flip=True,
                             rotation_range=30)
# prepare an iterators for each dataset
train_it = datagen.flow_from_dataframe(dataframe=train,
                                       directory=path, 
                                       class_mode='categorical',
                                       target_size=(224, 224),
                                       x_col="file",
                                       y_col="class_id",
                                       batch_size=2,
                                       seed=7) # set seed for stable performance

test_it = datagen.flow_from_dataframe(dataframe=test,
                                       directory=path,  
                                       class_mode='categorical',
                                       target_size=(224, 224),
                                       x_col="file",
                                       y_col="class_id",
                                       batch_size=2,
                                       seed=7)

In [5]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras import Sequential

In [6]:
model = Sequential()
model.add(Conv2D(32, (5,5),padding='valid',activation='relu',input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(2,activation='softmax'))

In [7]:
from keras.losses import CategoricalCrossentropy
from keras.optimizers import Adam
opti = Adam(learning_rate=0.0001)
model.compile(
  optimizer=opti,
  loss=CategoricalCrossentropy(),
  metrics=[tf.keras.metrics.Precision()])

In [ ]:
history = model.fit( 
  train_it,
  validation_data=test_it,
  epochs=6)

In [ ]:
# test on random pictures
from keras.utils import load_img
image = load_img('../orange_dataset/test/rotten/rotated_by_15_Screen Shot 2018-06-12 at 11.24.43 PM.png', target_size=(224, 224))
img = np.array(image)
img = img / 255.0
img = img.reshape(1,224,224,3)
label = model.predict(img)
print(label)